## Reducer: Sylvie Dagoret-Campagne

- creation date : May 30th 2016
- purpose : CCD reduction of Monocam data
- Analysis : Spectrometry

## Reviewer: Sylvie Dagoret-Campagne

# IPython notebook crash course

Click on a code cell (has grey background) then press Shift-Enter (at the same time) to run a code cell. That will add the controls (buttons, etc) you use to do the reduction one-by-one; then use them for reduction.

# reducer crash course

### Rule 0: Run the code cells in order

The world won't end if you break this rule, but you are more likely to end up with nonsensical results or errors. Incidentally, welcome to python indexing, which starts numbering at zero.

### Rule 1: Do not run this notebook in the directory containing your unreduced data

`reducer` will not overwrite anything but the idea is that you will keep a copy of this notebook with your reduced data.

### Rule 2: Keep the cells you need, delete the ones you don't

IPython notebooks are essentially customizable apps. If you don't shoot dark frames, for example, delete the stuff related to darks.

### Rule 3: If you find bugs, please report them

You can report bugs, make feature requests or (best of all) submit pull requests from [reducer's home on github](https:github.com/mwcraig/reducer)

### Bonus Pro Tip: Feel free to ignore the code in the code cells

Code is there so that people who know python can see what is going on, but if you don't know python you should still be able to use the notebook. Just remember to Shift-Enter on each code cell to run it, then fill in the form(s) that appear in the notebook.


In [1]:
from __future__ import division, print_function

%matplotlib inline

import numpy as np

import reducer.gui
import reducer.astro_gui as astro_gui
from reducer.image_browser import ImageBrowser
import msumastro

from reducer import __version__
print(__version__)

0.2.6


# Enter name of directory that contains your data in the cell below, or...

### ...leave it unchanged to try out reducer with low-resolution dataset

That low-resolution dataset will expand to about 300MB when uncompressed


In [2]:
# To use the sample data set:
data_dir = reducer.notebook_dir.get_data_path()

# Or, uncomment line below and modify as needed
data_dir = '2013-07-03-r-only'

destination_dir = '.'

# Type any comments about this dataset here

Double-click on the cell to start editing it.

# Load your data set

In [3]:
images = msumastro.ImageFileCollection(location=data_dir, keywords='*')

# Image Summary

## Includes browser and image/metadata viewer

This is not, strictly speaking, part of reduction, but is a handy way to take a quick look at your files.

In [4]:
tt = msumastro.TableTree(images.summary_info, ['imagetyp', 'exposure'], 'file')
fits_browser = ImageBrowser(tt, demo=False, directory=data_dir)
fits_browser.display()
fits_browser.padding = '10px'

### You can reconfigure the image browser if you want (or not)

By passing different keys into the tree constructor you can generate a navigable tree based on any keys you want.

In [5]:
tt2 = msumastro.TableTree(images.summary_info, ['filter', 'imagetyp', 'exposure'], 'file')
im_a_tree_too = ImageBrowser(tt2, demo=False, directory=data_dir)
im_a_tree_too.display()

# Make a master bias image

## Reduce the bias images

In [6]:
bias_reduction = astro_gui.Reduction(description='Reduce bias frames',
                                     toggle_type='button',
                                     allow_bias=False,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     apply_to={'imagetyp': 'bias'},
                                     destination=destination_dir)
bias_reduction.display()

## Combine bias images to make master bias

In [7]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')
bias_settings = astro_gui.Combiner(description="Master Bias Settings",
                                   toggle_type='button',
                                   file_name_base='master_bias',
                                   image_source=reduced_collection,
                                   apply_to={'imagetyp': 'bias'},
                                   destination=destination_dir)
bias_settings.display()

Master Bias Settings Yes
	Clip before combining? No
	Combine images? Yes
		Combination method:: Median
		Scale before combining? No
	Group by: No


TypeError: 'NoneType' object has no attribute '__getitem__'

# Make a master dark

## Reduce dark images

In [11]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')
dark_reduction = astro_gui.Reduction(description='Reduce dark frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'dark'})

dark_reduction.display()

## Combine reduced darks to make master(s)

Note the ``Group by`` option in the controls that appear after you execute the cell below. `reducer` will make a master for each value of the FITS keyword listed in ``Group by``. By default this keyword is named ``exposure`` for darks, so if you have darks with exposure times of 10 sec, 15 sec and 120 sec you will get three master darks, one for each exposure time.

In [ ]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')

dark = astro_gui.Combiner(description="Make Master Dark",
                          toggle_type='button',
                          file_name_base='master_dark',
                          group_by='exposure',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'dark'},
                          destination=destination_dir)
dark.display()

# Make master flats

## Reduce flat images

In [12]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')
flat_reduction = astro_gui.Reduction(description='Reduce flat frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=True,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'flat'})

flat_reduction.display()

## Build masters by combining

Again, note the presence of ``Group by``. If you typically use twilight flats you will almost certainly want to group by filter, not by filter and exposure.

In [13]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')

flat = astro_gui.Combiner(description="Make Master Flat",
                          toggle_type='button',
                          file_name_base='master_flat',
                          group_by='exposure, filter',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'flat'},
                          destination=destination_dir)
flat.display()

# Reduce the science images

There is some autmatic matching going on here:

+ If darks are subtracted a dark of the same edxposure time will be used, if available. If not, and dark scaling is enabled, the dark with the closest exposure time will be scaled to match the science image.
+ If the dark you want to scale appears not to be bias-subtracted an error will be raised.
+ Flats are matched by filter.

In [14]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')
light_reduction = astro_gui.Reduction(description='Reduce light frames',
                                      toggle_type='button',
                                      allow_bias=True,
                                      master_source=reduced_collection,
                                      allow_dark=True,
                                      allow_flat=True,
                                      input_image_collection=images,
                                      destination=destination_dir,
                                      apply_to={'imagetyp': 'light'})

light_reduction.display()

## Wonder what the reduced images look like? Make another image browser...

In [15]:
reduced_collection = msumastro.ImageFileCollection(location=destination_dir, keywords='*')

In [16]:
tt3 = msumastro.TableTree(reduced_collection.summary_info, ['imagetyp', 'exposure'], 'file')
fits_browser = ImageBrowser(tt3, demo=False, directory=reduced_collection.location)
fits_browser.display()

TypeError: First argument must be an astropy.table.Table instance